In [1]:
import scipy
import statistics
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib as mpl
import collections
from collections import defaultdict
import matplotlib
from matplotlib import pyplot as plt
import scipy.io as io
%matplotlib inline

matplotlib.rcParams['pdf.fonttype'] = 42

from _util_updated import nested_dict
from _util_updated import turn_point
from _util_updated import load_data
from _util_updated import filter_umi

In [8]:
sg_csv_dir = ''
sg_csv = (pd.read_csv(sg_csv_dir + 'All_sgRNA.csv', delimiter=',', header = 0))
sg_csv.columns = ('ID', 'Seq')

In [31]:
sgRNA_DIR = ''

DAA189_SG_RAW = pd.read_csv(sgRNA_DIR + 'DAA189_matrix_featurecount.csv.gz', compression='gzip', header=0,index_col = 0, sep=',', quotechar='"')
sg_seq = []
new_columns = []
for sg in DAA189_SG_RAW.index:
    sg_seq.append(sg.split('_')[2])
DAA189_SG_RAW.index = sg_seq
for col_id in DAA189_SG_RAW.columns:
    col_name_adj = col_id + '-1'
    new_columns.append(col_name_adj)
DAA189_SG_RAW.columns = new_columns

DAA190_SG_RAW = pd.read_csv(sgRNA_DIR + 'DAA190_matrix_featurecount.csv.gz', compression='gzip', header=0,index_col = 0, sep=',', quotechar='"')
sg_seq = []
new_columns = []
for sg in DAA190_SG_RAW.index:
    sg_seq.append(sg.split('_')[2])
DAA190_SG_RAW.index = sg_seq
for col_id in DAA190_SG_RAW.columns:
    col_name_adj = col_id + '-2'
    new_columns.append(col_name_adj)
DAA190_SG_RAW.columns = new_columns

In [32]:
DAA189_df_adj,cutoffs = filter_umi(DAA189_SG_RAW)

print(np.mean(cutoffs))
### Replace sgRNA Sequence with Name
temp_DAA189 = DAA189_df_adj
new_name = []
for seq in temp_DAA189.index:
        name_index = sg_csv[sg_csv.Seq == seq].index
        sgRNA_name = sg_csv.ID[name_index].tolist()[0]
        new_name.append(sgRNA_name)
temp_DAA189.index = new_name      
DAA189_df_name = temp_DAA189.T

2.63265306122449


In [33]:
DAA190_df_adj,cutoffs = filter_umi(DAA190_SG_RAW)
print(np.mean(cutoffs))

### Replace sgRNA Sequence with Name
temp_DAA190 = DAA190_df_adj
new_name = []
for seq in temp_DAA190.index:
        name_index = sg_csv[sg_csv.Seq == seq].index
        sgRNA_name = sg_csv.ID[name_index].tolist()[0]
        new_name.append(sgRNA_name)
temp_DAA190.index = new_name    
DAA190_df_name = temp_DAA190.T

2.4285714285714284


### Combine Matrix

In [34]:
All_df_adj = pd.concat([DAA189_df_name, DAA190_df_name]).fillna(value=0)

In [36]:
sg_cells = []

for cell in All_df_adj.index:
    if sum(All_df_adj.loc[cell]) > 0:
        sg_cells.append(cell)
All_df_adj_SG = All_df_adj.loc[sg_cells]

In [37]:
All_df_adj_SG.to_pickle('./Focused_SGRNA_IND_091121.pkl')

### Grouped Name (Group sgRNA for each target)

In [41]:
for column in All_df_adj_SG.columns:
    new_column = column.split('_')[0]
    All_df_adj_SG.rename(columns={column:new_column}, inplace = True)

In [42]:
All_df_adj_grouped = All_df_adj_SG.groupby(All_df_adj_SG.columns, axis=1).sum()

In [43]:
All_df_adj_grouped.to_pickle('./Focused_SGRNA_IND_091121_GROUP.pkl')